In [1]:
import func
import pandas as pd
import numpy as np

In [2]:
matrix_raw = func.load_matrix_tiny(1000) # 1000개 실행시 55초 # 5000개 실행시 12분
matrix = matrix_raw.copy()

load completed
Preprocessing completed


100%|██████████| 932/932 [00:02<00:00, 368.86it/s]


Ingredient split completed


100%|██████████| 929/929 [00:00<00:00, 1726.04it/s]


Matrix creation completed
recipe X food matrix is saved with the namematrix/Dec 05 15:41


In [159]:
# 한번 쓰인 식재료(칼럼) 제외
def not_only_one(df):
    column_value_counts  = df.nunique(axis=0)
    column_names = column_value_counts[column_value_counts >= 2].index
    df_not_only_one_col = df[column_names].copy()
    print("칼럼 수 변화: " + f"{df.shape[1]} -> {df_not_only_one_col.shape[1]}")

    df_not_only_one_row = df_not_only_one_col[df_not_only_one_col.nunique(axis =1) >= 2].copy()
    print("행의 수 변화: " + f"{df_not_only_one_col.shape[0]} -> {df_not_only_one_row.shape[0]}")
    return  df_not_only_one_row

In [163]:
matrix_test = matrix_raw.copy()
matrix_test.nunique(axis=0)

recipe_title    4770
무우소                2
냉동사각유부             2
스파이스가루혹은강황가루       1
습식맵쌀가루             2
                ... 
토장                 2
양파약간씩              1
홍게간장또는간장           2
고추장푹퍼서             1
올리브약간              1
Length: 5625, dtype: int64

In [160]:
matrix_selected = not_only_one(matrix)

칼럼 수 변화: 5625 -> 3819
행의 수 변화: 4817 -> 4817


In [161]:
def minmax_by_col(df):
    if 'recipe_title' in df.columns :
        df = df.drop('recipe_title', axis =1).copy()
    else : 
        pass
    for col in df.columns:
        data = df[col]
        min = data.min()
        max = data.max()
        df[col] = (data-min)/(max-min)
    return df

In [162]:
matrix_scaled = minmax_by_col(matrix_selected) # 5000개 레시피 : 1분 30초

In [164]:
from scipy import linalg
U, S, V = linalg.svd(matrix_scaled.values)
(U[:,:100].shape, np.diag(S[:100]).shape, V[:, :100].shape)

((4817, 100), (100, 100), (3818, 100))

In [165]:
from scipy import linalg
def matrix_decomposition(df, n = 10):
    U, S, V = linalg.svd(df.values)
    recipe_represented = U[:, :n]@np.diag(S[:n])
    ingredient_represented = V[:, :n]@np.diag(S[:n])
    print(f"{df.shape[0]}개의 레시피, {df.shape[1]}개의 식재료 -> {n}차원으로 재표현 완료")
    return recipe_represented, ingredient_represented


In [166]:
recipe2, ingred2 = matrix_decomposition(matrix_scaled, n = 100) #5000개 42초

4817개의 레시피, 3818개의 식재료 -> 100차원으로 재표현 완료


In [178]:
recipe2.shape

(4817, 100)

In [176]:
matrix['recipe_title']

0                                     호박 새우젓찌개 만드는 방법~★
1                                          바글바글 전복 된장찌개
2                     순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!
3                                         정통 바지락 순두부 찌개
4                                           크림토마토 비프 스튜
                             ...                       
4812                    구수한 들깨수제비. 추운 날씨에 만두피로 간편하게 뚝딱.
4813                                       대한민국 일품식 불고기
4814                                           채소 단호박탕수
4815                 집에서 만드는 인도식 마크니커리 진짜 맛있는 새우 마늘쫑 조합
4816    #제쳘요리 #봄쭈꾸미요리 #쭈꾸미샤브샤브만들기 #시금치 #된장육수 #봄쭈꾸미로체력보강
Name: recipe_title, Length: 4817, dtype: object

In [182]:
ingred2[0]

array([2.38021629e-02, 1.39273075e-03, 1.36286865e-03, 7.99647046e-03,
       3.85273144e-03, 1.43161656e-05, 2.58405005e-02, 6.03877721e-04,
       7.83419399e-06, 3.47968757e-03, 6.20667439e-03, 7.38106511e-03,
       5.57845595e-02, 4.16302132e-02, 6.61631860e-05, 3.44652438e-04,
       8.62955355e-03, 6.27504977e-04, 1.81366826e-02, 1.63159338e-02,
       1.97841694e-03, 3.24744090e-05, 4.40203072e-02, 3.65107047e-02,
       2.96786087e-07, 1.06833374e-03, 3.46789749e-03, 3.01188483e-03,
       1.04677808e-05, 1.69729385e-02, 2.58314541e-05, 6.98731242e-05,
       2.16161461e-02, 4.36069588e-03, 1.44511991e-03, 1.38488406e-02,
       2.27153491e-03, 1.77794952e-02, 1.70415563e-02, 2.38626965e-04,
       1.68248756e-03, 1.58309122e-02, 6.07054189e-05, 2.67140315e-05,
       2.66404060e-02, 1.33917177e-06, 1.13576013e-02, 2.30886708e-02,
       1.53245953e-03, 4.98196950e-04, 4.48049027e-03, 5.15294821e-03,
       2.97884061e-03, 4.83470237e-02, 3.79785258e-03, 7.89972490e-03,
      

In [179]:
recipe2[0]

array([ 1.37446151e-01, -1.80759359e-02, -4.64222517e-02,  1.67831930e-01,
       -1.67992576e-01,  1.29932990e-02,  8.13904489e-02,  2.77335479e-02,
        1.09208581e-02, -2.87024342e-03,  7.17130908e-02, -2.68172904e-01,
       -6.10200575e-02,  1.32633881e-01, -2.51807605e-01,  2.20092742e-01,
        2.98385756e-01,  7.89248033e-01,  2.00939108e-01,  4.82347834e-02,
       -7.95425843e-02, -1.75949077e-03, -3.62729947e-02,  6.50802210e-02,
        2.36172116e-02,  3.31652141e-02, -5.83397821e-04, -9.17423135e-03,
       -4.73483977e-02,  1.27470140e-02, -2.30268427e-02,  1.92536869e-02,
       -2.82789507e-02, -1.08268962e-02,  2.54609018e-02,  3.02203429e-03,
        7.75765113e-03,  8.91112637e-03,  7.61401469e-03,  7.83174053e-03,
        5.89637104e-03, -2.58535083e-02, -4.20440617e-04, -1.21986185e-02,
        1.70762291e-02, -3.74474504e-03, -1.71043588e-02, -1.76492102e-02,
        5.69355434e-03, -4.27672416e-03,  8.12306431e-04, -1.78094555e-03,
        1.00217413e-03, -